# HOW TO: Generate Fake Data with R (Using Jupyter Notebook)
`--------------------------------------------------`

**___________________________Featuring: Mr Fugu Data Science____**


[Find me on Github](https://github.com/MrFuguDataScience)

`----------------------------------------------------------`

`It appears that while I have packages loaded into R-Studio, do not coincide with Jupyter Notebook. I tried to search Anaconda installs but could not find them. Hence, we are doing this, if it becomes a habit I will create a function call.`

*Also, `Tidyverse` takes several minutes alone due to dependencies*

[rpython](https://cran.r-project.org/web/packages/rPython/index.html)| [reticulate](https://cran.r-project.org/web/packages/reticulate/vignettes/calling_python.html) | [RPostgreSQL](https://cran.r-project.org/web/packages/RPostgreSQL/index.html) | [dbConnect](https://www.rdocumentation.org/packages/DBI/versions/0.5-1/topics/dbConnect) | [randomNames](https://cran.r-project.org/web/packages/randomNames/vignettes/randomNames.html)

In [ ]:
# install.packages("generator")
# install.packages("dbConnect")
# install.packages("tidyverse")
# install.packages("rlang")
# install.packages("RPostgreSQL")
# install.packages("rPython")
# install.packages("randomNames")
# install.packages("reticulate")

In [1]:
library(RMySQL) # interface R->Mysql
library(RPostgreSQL) # run psql instance
library(DBI) # R database interface

library(generator) # fake personal information
library(randomNames)  # [this is not updated in R-studio] as of Apr 1,2020 but works here :) 

library(knitr) # help run code
library(markdown) # create markdown files i.e. pdf
library(dbConnect) # connect to databases
library(tidyverse) # if you parse files
library(dplyr)
library(rPython)  # Run Python code, make function calls, assign and retrieve variables, etc. from R.
library(randomNames)  #[this is not updated in R-studio] as of Apr 1,2020 but works here :) 
library(reticulate)  # interface with python modules,classes, functions

Loading required package: DBI

Attaching package: ‘RPostgreSQL’

The following object is masked from ‘package:RMySQL’:

    dbApply

Loading required package: gWidgets
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  3.0.0     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::id()     masks gWidgets::id()
✖ dplyr::lag()    masks stats::lag()
Loading required package: RJSONIO


In [2]:
#Users info:

crdt_crd<-generator::r_credit_card_numbers(n=6000)
email<-generator::r_email_addresses(n=6000)

dob<-generator::r_date_of_births(n=6000)

first_name<-randomNames(n=6000,return.complete.data = FALSE,which.names = "first")
last_name<-randomNames(n=6000,return.complete.data = FALSE,which.names = "last")
primary_phone_number<-generator::r_phone_numbers(n=6000)


In [3]:
user_info<-list(crdt_crd,email,first_name,last_name,primary_phone_number)
data.frame(matrix((user_info),nrow=length(user_info)))


matrix..user_info...nrow...length.user_info..


In [4]:
users<-as.data.frame(matrix(unlist(cbind(user_info)),ncol = 5),as.factor=FALSE)
users<-users[0:5826,]
# users
names(users)[1]<-paste("credit_card")
names(users)[2]<-paste("email")
names(users)[3]<-paste("first_name")
names(users)[4]<-paste("last_name")
names(users)[5]<-paste("primary_phone_number")
head(users)

credit_card,email,first_name,last_name,primary_phone_number
7929-3849-5090-8440,wls@mlatgw.crw,Zackarie,el-Sami,7956729362
7606-6195-2000-3181,bawclqrmd@hofcrm.bgt,Andrea,Luna Gonzalez,3172817425
1201-6802-9166-1923,ptbvkeix@pfabgnr.jlx,Austin,Beasley,8359345398
5314-3457-2009-3950,ujdfimo@yjdavqmboi.wib,Sierra,Wall,2651678629
4754-4613-1733-6930,cmzopsi@izyepuxf.twk,Veronica,Castillo-Mosqueda,4161376592
4519-2441-3413-2271,qsok@qgfsvbtl.beg,Lacey,Del Valle,8754633287


# Before we proceed: there is a caveat and I need to discuss, when creating random (pseudo) random data you need to look for duplicates.

+ There are duplicate names (first and last)
    + `I AM ONLY CONCERNED IF THE PAIR OF NAMES COMES UP MORE THAN 1 TIME`:

In [5]:
# Here is an issue when you create random data!

sum(duplicated(users$credit_card))
sum(duplicated(users$email))
sum(duplicated(users$last_name))
sum(duplicated(users$primary_phone_number))
sum(duplicated(users$first_name))

[1] 0

[1] 0

[1] 2253

[1] 0

[1] 3080

# There are 13 matches to (first,last) names which need to be removed from our database! 
+ It will depend what we want as a qualifier, we can keep them and base our data off of an `ID` which will be the `primary key` for a `RDMS`, and figure out what to use for a `NoSQL`. 

+ While this can create problems, you have to understand people have same names and are different people and need to establish what other data will coincide causing issues. 

In [6]:
# users %>% group_by(first_name, last_name) %>% summarize(time = max(time, na.rm = T))
users[duplicated(users[c(3, 4)]),] 
sum(duplicated(users[,3:4]))

# users[duplicated(users[,3:4]),]


,credit_card,email,first_name,last_name,primary_phone_number
1107,1928-6110-8620-6330,twmqxkjs@zhi.thi,Haley,Griffin,9164763956
2024,7537-7336-7923-9624,ozvkleg@pxy.biu,Michael,Johnson,8277136952
3013,1913-9167-9436-7875,t@fosbrhqalt.ibk,Jessie,Garcia,9268128749
3962,6240-3863-7337-9460,zgvr@ndktolgfj.zpw,Joshua,Smith,5982644235
4336,9430-2184-4723-3507,tjvaiqlb@gnbtikvf.nwp,Dylan,Martinez,8495643725
4481,3880-4819-3080-2732,nbgt@v.yvj,Kevin,Smith,8295988492
4593,6137-2236-1904-6976,gcb@kd.das,Michael,Johnson,9484867694
4693,5524-8466-4376-9749,optsuywzfc@pi.nos,Nathan,Mills,3975613645
4769,1297-9047-8685-6629,baxkonlewi@fbejaxdkmg.wyj,Caitlin,al-Farha,5811594586
5476,6936-4627-8344-1920,pzqinh@izujxda.lzm,Sarah,Johnson,2892812715


[1] 10

In [7]:
# I am accessing Python module [OS] and calling with R formatting:

#os <- import("os")
#os$listdir(".")
# write(users,'fake_users.csv')
# typeof(users)
# 
# data.frame(matrix((users),nrow=length(users)))
# usr<-as.data.frame(matrix(unlist(cbind(users)),ncol = 5),as.factor=FALSE)

# is.data.frame(usr)
# is.list(usr)
tail(users)

,credit_card,email,first_name,last_name,primary_phone_number
5821,5919-3854-9500-1111,nl@iuheoxpvqs.afb,John,Brown,5474136951
5822,1218-1082-3695-4762,cur@kbrcaexyn.xzv,Elizabeth,Smedra,9152387452
5823,7779-5179-3157-1687,a@euhfa.hxi,Mayah,Zhen,2949766138
5824,1413-3747-6578-4140,rfctajiuy@yrgfaud.gei,Sunshine,Prinster,6312419514
5825,6005-8095-7699-6840,ltcgr@gdwvmz.mhp,Sarah,Runnels,9642465617
5826,1796-1349-5148-5405,ujmnxb@iwqfp.kdj,Stefan,Andres,9287145739



# Time to Dance because we are going to send these data from R --> Mysql

In [ ]:
# **Run when getting to this line:**
# <!-- ![dance_time](giphy.gif) -->

In [8]:
W_O<-write.csv(users,'fake_users_opR.csv')


`----------------------------------------------------`

# Section Sending `R` data to `MySQL`:

[official RmySQL pdf](https://cran.r-project.org/web/packages/RMySQL/RMySQL.pdf)

# open terminal to enter MySQL:
+ I am creating a new user instead of using `root` for this activity
    + then I will create a new database that I will send data to
    + from there we will send data to MySQL and then perform `queries` from `R --> MySQL` 
    
 **Type:** if using Mac homebrew installation: 
 
 `mysql.server start` then press enter
 
 `mysql -uroot -phere_type_your_password` press enter

now you will be in the mysql server:

`CREATE DATABASE your_database_name`;

`CREATE USER` `'user_name_you_create'`@`'localhost'` IDENTIFIED BY `'password_you_create';`

`GRANT ALL PRIVILEGES ON` `database_you_create.*` `TO` `'user_name_you_create'`@`'localhost';`

`FLUSH PRIVILEGES;`
 

`______________________________________________________________________________________`

# Possible Errors: when using R->MySQL

if you see: `loading local data error`:  
+ go to your *root* login and enter this after login

`SHOW GLOBAL variables like 'local_infile';`


| Variable_name 	| Value 	|
|---------------	|-------	|
| local_infile  	| off   	|

<center>1 row in set (0.00 sec)</center>


`SET GLOBAL local_infile=ON;`

*Query OK, 0 rows affected (0.00 sec)*

-------Now you will be able to transer data to and from, log out of *root* and back into your user_name--------

# In MySQL:

`CREATE TABLE IF NOT EXISTS user_info(credit_card varchar(19),
    email varchar(36),first_name char(20),last_name varchar(24),
    primary_phone_number INT(10),id INT AUTO_INCREMENT PRIMARY KEY);`
    
    
    
*OUTPUT:*


| Field                	| Type        	| Null 	| Key 	| Default 	| Extra          	|
|----------------------	|-------------	|------	|-----	|---------	|----------------	|
| credit_card          	| varchar(19) 	| Yes  	|     	| Null    	|                	|
| email                	| varchar(36) 	| Yes  	|     	| Null    	|                	|
| first_name           	| char(20)    	| Yes  	|     	| Null    	|                	|
| last_name            	| varchar(24) 	| Yes  	|     	| Null    	|                	|
| Primary_phone_number 	| int         	| Yes  	|     	| Null    	|                	|
| id                   	| int         	| No   	| PRI 	| Null    	| auto-increment 	|

In [ ]:
# head(users)

In [9]:
drv<-dbDriver("MySQL") 

In [10]:
connection<-dbConnect(MySQL(), dbname = 'project_practice', host='localhost',user='your_info',
                      password='your_password')


In [11]:
dbGetInfo(connection)
myquery<-"show databases;"
dbGetQuery(connection,myquery)
#dbListTables()

$host
[1] "localhost"

$user
[1] "MrFugu"

$dbname
[1] "project_practice"

$conType
[1] "Localhost via UNIX socket"

$serverVersion
[1] "8.0.19"

$protocolVersion
[1] 10

$threadId
[1] 14

$rsId
list()

Database
information_schema
project_practice


# Sending data R->Mysql as  [data.frame()] format


+ overwrite=TRUE, append=FALSE   (*create db no matter what*)

+ overwrite=FALSE,append=TRUE    (*only append*)

In [12]:
# Inserting data into MySQL table:
dbWriteTable(conn=connection,overwrite=TRUE,append=FALSE,value=data.frame(users),name='users_info')


[1] TRUE

In [13]:
dbListConnections(drv=RMySQL::MySQL())
dbIsValid(connection)

[[1]]
<MySQLConnection:0,0>


[1] TRUE

# Sending Requests Distinctions:

+ **dbSendQuery()**: gets query and **doesn't** grab data
    + **dbFetch()** takes data and must call `dbClearResult()` after
        + **dbGetQuery()**: by default calls `dbSendQuery()` followed by `dbFetch()` allowing data to be 
            free from `dbClearResult()` 
            
[R_doc_explaining](https://www.rdocumentation.org/packages/DBI/versions/0.5-1/topics/dbSendQuery)

In [15]:
myquery<-paste("SELECT * FROM users_info LIMIT 5;")
dbGetQuery(connection,myquery)

row_names,credit_card,email,first_name,last_name,primary_phone_number
1,7929-3849-5090-8440,wls@mlatgw.crw,Zackarie,el-Sami,7956729362
2,7606-6195-2000-3181,bawclqrmd@hofcrm.bgt,Andrea,Luna Gonzalez,3172817425
3,1201-6802-9166-1923,ptbvkeix@pfabgnr.jlx,Austin,Beasley,8359345398
4,5314-3457-2009-3950,ujdfimo@yjdavqmboi.wib,Sierra,Wall,2651678629
5,4754-4613-1733-6930,cmzopsi@izyepuxf.twk,Veronica,Castillo-Mosqueda,4161376592


In [16]:
myquery_<-paste("SELECT * FROM users_inf
WHERE first_name='Samuel'
LIMIT 3;")
dbGetQuery(connection,myquery_)

# dbDisconnect(conn=connection)   # end connection

row_names,credit_card,email,first_name,last_name,primary_phone_number
471,6344-9809-1636-3006,rohjtxlywq@eczwlayk.rck,Samuel,Domingo,6517387564
1102,4482-1704-1874-1477,s@gahsbfodi.qyn,Samuel,el-Aboud,5686827839
1127,8609-2473-7697-6345,cu@hobs.vtx,Samuel,Clark,1382536518


In [17]:
#shows what fields are in this TABLE:
dbListFields(connection, 'users_inf')


[1] "row_names"            "credit_card"          "email"               
[4] "first_name"           "last_name"            "primary_phone_number"

In [18]:
# Taking data from MySQL: and storing as [data]

rs = dbSendQuery(connection,myquery_) # sends query
data = fetch(rs)   # grabs data from query
dbClearResult(rs)  # need this line


# OPTIONS:
# d1 <- dbFetch(rs, n = 10) # extract data in chunks of 10 rows
# d2 <- dbFetch(rs, n = -1) # extract all remaining data

[1] TRUE

In [19]:
data

row_names,credit_card,email,first_name,last_name,primary_phone_number
471,6344-9809-1636-3006,rohjtxlywq@eczwlayk.rck,Samuel,Domingo,6517387564
1102,4482-1704-1874-1477,s@gahsbfodi.qyn,Samuel,el-Aboud,5686827839
1127,8609-2473-7697-6345,cu@hobs.vtx,Samuel,Clark,1382536518


In [ ]:
# dbClearResult(dbListResults(connection)[[1]])
dbDisconnect(conn = connection)

In [ ]:
# dbReadTable()
# dbRemoveTable()

`----------------------------------------------------------------`

# Send `[.CSV] from R to MySQL`:

In [20]:
# dbWriteTable(conn=connection,project_practice, value=W_O,name='tempTable', row.names=FALSE,
#              field.types=list(credit_card="VARCHAR(19)",email="VARCHAR(36)",
#                               first_name="CHAR(20)", 
#                               last_name="VARCHAR(24)",primary_phone_number="INT(10)"), 
#                               append=FALSE, overwrite=TRUE)

WP<-'/Users/zatoichi59/Desktop/Projects/fake_users_opR.csv'
dbWriteTable(conn=connection,value=WP, name='user_o',append=FALSE,
             overwrite=TRUE,row.names=FALSE)



# sum(is.na(users))
# CREATE TABLE IF NOT EXISTS user_info(credit_card varchar(19),emai varchar(36),first_name char(20),
#                                      last_name varchar(24),id INT AUTO_INCREMENT PRIMARY KEY);

[1] TRUE

In [21]:
# myqry_<-paste("SELECT * FROM user_o
# WHERE first_name='Ivan';")
# dbGetQuery(connection,myqry_)

myquery<-paste("SELECT * FROM user_o LIMIT 5;")
dbGetQuery(connection,myquery)

X,credit_card,email,first_name,last_name,primary_phone_number
1,7929-3849-5090-8440,wls@mlatgw.crw,Zackarie,el-Sami,7956729362
2,7606-6195-2000-3181,bawclqrmd@hofcrm.bgt,Andrea,Luna Gonzalez,3172817425
3,1201-6802-9166-1923,ptbvkeix@pfabgnr.jlx,Austin,Beasley,8359345398
4,5314-3457-2009-3950,ujdfimo@yjdavqmboi.wib,Sierra,Wall,2651678629
5,4754-4613-1733-6930,cmzopsi@izyepuxf.twk,Veronica,Castillo-Mosqueda,4161376592


In [22]:
uu<-paste("ALTER TABLE user_o
ADD id INT AUTO_INCREMENT PRIMARY KEY;")

dbGetQuery(connection,uu)

<0 x 0 matrix>

In [23]:
myquery<-paste("SELECT * FROM user_o LIMIT 5;")
dbGetQuery(connection,myquery)

X,credit_card,email,first_name,last_name,primary_phone_number,id
1,7929-3849-5090-8440,wls@mlatgw.crw,Zackarie,el-Sami,7956729362,1
2,7606-6195-2000-3181,bawclqrmd@hofcrm.bgt,Andrea,Luna Gonzalez,3172817425,2
3,1201-6802-9166-1923,ptbvkeix@pfabgnr.jlx,Austin,Beasley,8359345398,3
4,5314-3457-2009-3950,ujdfimo@yjdavqmboi.wib,Sierra,Wall,2651678629,4
5,4754-4613-1733-6930,cmzopsi@izyepuxf.twk,Veronica,Castillo-Mosqueda,4161376592,5


# Citations:

https://stackoverflow.com/questions/42459423/cannot-install-r-packages-in-jupyter-notebook

https://stackoverflow.com/questions/6081439/changing-column-names-of-a-data-frame

https://cran.r-project.org/web/packages/reticulate/vignettes/calling_python.html

https://giphy.com/gifs/dance-alien-ufo-Z9WQLSrsQKH3uBbiXq (NO Copyrights!)

https://programminghistorian.org/en/lessons/getting-started-with-mysql-using-r

https://jagg19.github.io/2019/05/mysql-r/

https://cran.r-project.org/doc/manuals/r-devel/R-data.html#DBI

https://dba.stackexchange.com/questions/48751/enabling-load-data-local-infile-in-mysql

https://stackoverflow.com/questions/33860738/r-dbwritetable-or-a-mysql-delete-query


In [ ]:
# statementRes <- DBI::dbSendStatement(conn = connection,
#                      "CREATE TABLE IF NOT EXISTS user_info (
#                         credit_card varchar(19),
#                         email varchar(36),first_name char(16),last_name varchar(24),
#                         primary_phone_number int(10),id INT AUTO_INCREMENT PRIMARY KEY;")
# DBI::dbGetRowsAffected(statementRes)
# DBI::dbClearResult(statementRes)
# library(RODBC)
# channel <- odbcConnect("project_practice")
# sqlSave(channel, dat = usr, tablename = "user_info", rownames = FALSE, append = 
# TRUE)

# https://stackoverflow.com/questions/4084028/how-to-close-resultset-in-rmysql



data = fetch(rs, n=-1)

# This saves the results of the query as a data frame object. The n in the function specifies the number of records to retrieve, using n=-1 retrieves all pending records.

In [ ]:
getwd()
'/Users/zatoichi59/Desktop/Projects/fake_users_opR.csv'